In [1]:
import numpy as np 
import pandas as pd 
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import requests
from datetime import datetime

<ipython-input-1-ddc64e43fe1f>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook
C:\Users\kfuan\Anaconda3\envs\dl_fastai\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
stocks_list = pd.read_csv("../data/sp_500_stocks.csv")

In [4]:
stocks_list.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [5]:
momentum_portfolio = stocks_list.copy()

In [6]:
def chunks(input_list, n):
    for i in range(0, len(input_list), n):
        yield input_list[i: i+n]

In [7]:
batch_symbol = [name for name in chunks(stocks_list['Ticker'].values, 100)]

In [8]:
batch_strings = []
for batch in batch_symbol:
    batch_strings.append(",".join(batch))

In [9]:
momentum_portfolio[['Price', '1-Year change', 'Amount of shares holding']] = None

In [ ]:
def batch_api_call(symbols):
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=quote,stats&filter=latestPrice,year1ChangePercent&token={IEX_CLOUD_API_TOKEN}"
    return batch_data = requests.get(batch_api_url).json()

In [11]:
def get_price(x):
    return x['quote']['latestPrice']
    
def get_1Ychange(x):
    return x['stats']['year1ChangePercent']

In [12]:
vec_get_price = np.vectorize(get_price)
vec_get_1Ychange = np.vectorize(get_1Ychange)

In [13]:
d0 = datetime.now()

start = 0
for batch in tqdm_notebook(batch_strings):
    data = batch_api_call(batch)
    data_array = np.array(list(data.values()))
    prices = vec_get_price(data_array)
    oneYchange = vec_get_1Ychange(data_array)
    
    momentum_portfolio['Price'][start: start+len(data_array)] = prices
    momentum_portfolio['1-Year change'][start: start+len(data_array)] = oneYchange
    
    start += len(data_array)
    
print(datetime.now() - d0)


0:00:06.187405


In [14]:
momentum_portfolio

,Ticker,Price,1-Year change,Amount of shares holding
0,A,128.87,0.500148,None
1,AAL,17.6,-0.393794,None
2,AAP,159.98,0.215683,None
3,AAPL,139.502,0.757895,None
4,ABBV,108.74,0.360337,None
...,...,...,...,...
500,YUM,103.76,-0.0170241,None
501,ZBH,167.2,0.0350985,None
502,ZBRA,409.78,0.667376,None
503,ZION,48.24,0.0741392,None


# Equally weighted top 50 returns

In [43]:
equal_weight_momentum = momentum_portfolio.copy()
equal_weight_momentum.sort_values(by='1-Year change', ascending=False, inplace=True)
equal_weight_momentum = equal_weight_momentum[:50]
equal_weight_momentum.reset_index(drop=True, inplace=True)

In [52]:
equal_weight_momentum.head()

,Ticker,Price,1-Year change,Amount of shares holding
0,CARR,40.09,2.29205,None
1,FCX,30,1.50582,None
2,ALGN,643.45,1.43007,None
3,PYPL,281.56,1.27994,None
4,NVDA,565.75,1.22321,None


In [72]:
def equal_weight_position_size(n):
    global portfolio_size
    while True:
        try:
            portfolio_size = float(input("Enter the size of your portfolio: "))
            
            break
        except:
            print('Please input a correct number format.')
            continue
    global position_size
    position_size = portfolio_size/n

In [79]:
equal_weight_position_size(equal_weight_momentum.shape[0])

Enter the size of your portfolio: 10000000


In [80]:
print(f"size of total portfolio: {portfolio_size} \nsize of each position: {position_size}")

size of total portfolio: 10000000.0 
size of each position: 200000.0


In [57]:
equal_weight_momentum.loc[:, 'Amount of shares holding'] = np.floor(position_size / equal_weight_momentum.loc[:, 'Price'])

C:\Users\kfuan\Anaconda3\envs\dl_fastai\lib\site-packages\pandas\core\indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [59]:
equal_weight_momentum.head(10)

,Ticker,Price,1-Year change,Amount of shares holding
0,CARR,40.09,2.29205,4988
1,FCX,30,1.50582,6666
2,ALGN,643.45,1.43007,310
3,PYPL,281.56,1.27994,710
4,NVDA,565.75,1.22321,353
5,LB,48.96,1.06168,4084
6,SIVB,511.45,0.939616,391
7,PWR,77.59,0.924055,2577
8,DE,315.79,0.9203,633
9,CDNS,141.6,0.847221,1412


In [61]:
equal_weight_momentum.to_excel('equal_weight_top50winners.xlsx', 
                               sheet_name='equally-weighted', 
                               index=False)